### Télécharger des bases de données à partir de SPARQL

In [1]:
import warnings

import pandas as pd
from IPython.core.display import display
from IPython.display import Markdown, clear_output
from ipywidgets import Layout, Box, VBox, widgets

warnings.filterwarnings("ignore")

ENDPOINT: str = "http://kaiko.getalp.org/sparql"  #"https://statistics.gov.scot/sparql"

In [2]:
%run ../../src/Dataset_tools

In [3]:
list_datasets = get_datasets(ENDPOINT, verbose=False)  # List of all datasets available
list_datasets_short = pd.concat({'dataset': list_datasets['dataset'].map(lambda x: x.split('/')[-1]), 'commentaire': list_datasets['commentaire']})

DataFrame = pd.DataFrame()
output = widgets.Output()

dataset_user_choice = widgets.Dropdown(
    options=[(name, full_name) for name, full_name in zip(list_datasets_short['dataset'].values, list_datasets['dataset'].values)],
    description="Choix:", layout=Layout(flex='1 3 auto', width='auto')
)
user_choice_confirm = widgets.Button(description='Soumettre', icon='check', layout=Layout(flex='1 1 auto', width='auto'),
                                     tooltip="Cliquer ici pour confirmer votre choix"
                                    )

description = widgets.Label(
    value=f"Description: {list_datasets[list_datasets['dataset'] == dataset_user_choice.value]['commentaire'].values[0]}",
    layout=Layout(flex='1 1 auto', width='auto'))

box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='90%')

line_1 = Box(children=[dataset_user_choice, user_choice_confirm], layout=box_layout)
line_2 = Box(children=[description], layout=box_layout)
ui = VBox([line_1, line_2])


def user_choice_change(obj):
    description.value = f"Description: {list_datasets[list_datasets['dataset'] == dataset_user_choice.value]['commentaire'].values[0]}"


def user_choice_confirm_eventhandler(obj):
    with output:
        clear_output()
        output2 = widgets.Output()
        categories_long = get_features(ENDPOINT, dataset_user_choice.value)
        categories_short = categories_long["property"].map(lambda x: x.split('#')[-1].split('/')[-1]).to_frame()

        display(Markdown(data="#### Les différentes propriétés de ce dataset sont: "), categories_short)

        select = widgets.SelectMultiple(options=[(name, full_name) for name, full_name in zip(categories_short.values.reshape(-1),
                                        categories_long.values.reshape(-1))], description='Critères: ',
                                        layout=Layout(flex='1 1 auto', width='auto', height="auto"), style={'description_width': 'initial'})
        
        select_confirm = widgets.Button(description='Soumettre', icon='check', layout=Layout(flex='1 1 auto', width='auto'),
                                        tooltip="Cliquer ici pour confirmer votre choix")
        
        select_box = Box(children=[select], layout=box_layout)
        select_confirm_box = Box(children=[select_confirm], layout=box_layout)
        select_ui = VBox([select_box, select_confirm_box])

        def selection_confirm_eventhandler(obj):
            with output2:
                if len(select.value) >= 2:  # We will constuct the query
                    global DataFrame
                    clear_output()
                    DataFrame = download_dataset(ENDPOINT, dataset_user_choice.value, select.value)
                    display(DataFrame)

        select_confirm.on_click(selection_confirm_eventhandler)
        display(Markdown("####  Veuillez choisir au moins deux critères à télécharger: "), select_ui, output2)


user_choice_confirm.on_click(user_choice_confirm_eventhandler)
dataset_user_choice.observe(user_choice_change, 'value')
display(Markdown(data="#### Veuillez choisir un DataSet à étudier: "), ui, output)

IntProgress(value=0, bar_style='success', description='Loading:')

#### Veuillez choisir un DataSet à étudier: 

Output()